<a href="https://colab.research.google.com/github/aryab2003/GreenGo/blob/main/PlantDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)

In [ ]:
import os
import json
from zipfile import ZipFile
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!pip install

In [ ]:
!kaggle datasets download -d emmarex/plantdisease

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 97% 641M/658M [00:11<00:00, 95.0MB/s]
100% 658M/658M [00:11<00:00, 60.2MB/s]


In [ ]:
dataset= ZipFile('/content/plantdisease.zip')
dataset.extractall()
print(os.listdir('/content/PlantVillage'))

['Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_healthy', 'Pepper__bell___Bacterial_spot', 'Potato___healthy', 'Tomato_Early_blight', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Potato___Early_blight', 'Tomato__Tomato_mosaic_virus', 'Pepper__bell___healthy', 'Tomato_Bacterial_spot', 'Tomato__Target_Spot', 'Tomato_Septoria_leaf_spot', 'Potato___Late_blight']


In [ ]:
img_size=224
batch_size=32

In [ ]:
data_gen= ImageDataGenerator(rescale=1./255,
                            validation_split=0.2)

In [ ]:
train_generator= data_gen.flow_from_directory('/content/PlantVillage',
                                            target_size=(img_size,img_size),
                                            batch_size=batch_size,
                                            subset='training',
                                              class_mode='categorical'   )
test_generator= data_gen.flow_from_directory('/content/PlantVillage',
                                            target_size=(img_size,img_size),
                                            batch_size=batch_size,
                                            subset='validation',
                                              class_mode='categorical' )

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.


In [ ]:
from tensorflow.keras import regularizers
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(img_size,img_size,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(train_generator.num_classes,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,steps_per_epoch=train_generator.samples // batch_size,epochs=5,validation_data=test_generator,validation_steps=test_generator.samples // batch_size)

Epoch 1/5
516/516 [==============================] - 50s 86ms/step - loss: 1.1260 - accuracy: 0.6734 - val_loss: 0.5495 - val_accuracy: 0.8188
Epoch 2/5
516/516 [==============================] - 40s 77ms/step - loss: 0.3370 - accuracy: 0.8875 - val_loss: 0.5143 - val_accuracy: 0.8276
Epoch 3/5
516/516 [==============================] - 41s 79ms/step - loss: 0.1442 - accuracy: 0.9533 - val_loss: 0.4467 - val_accuracy: 0.8667
Epoch 4/5
516/516 [==============================] - 44s 86ms/step - loss: 0.0564 - accuracy: 0.9822 - val_loss: 0.4693 - val_accuracy: 0.8635
Epoch 5/5
516/516 [==============================] - 42s 82ms/step - loss: 0.0707 - accuracy: 0.9779 - val_loss: 0.5847 - val_accuracy: 0.8542


In [ ]:
loss,accuracy=model.evaluate(test_generator)
print('test accuracy:',accuracy)

129/129 [==============================] - 7s 56ms/step - loss: 0.5876 - accuracy: 0.8537
test accuracy: 0.8537117838859558


In [ ]:
def load_pre_process(image_path,target_size=(224,224)):
  img=Image.open(image_path)
  img=img.resize(target_size)
  img=np.array(img)
  img=np.expand_dims(img,axis=0)
  img=img.astype('float32')/255.0
  return img

In [ ]:
def predict_class_names(model,image_path,class_indices):
  img=load_pre_process(image_path)
  pred=model.predict(img)
  predicted_class=np.argmax(pred,axis=1)[0]
  predicted_names=class_indices[predicted_class]
  return predicted_names

In [ ]:
class_indices={v:k for k,v in train_generator.class_indices.items()}
json.dump(class_indices,open('class_indices.json','w'))

In [ ]:
img='/content/PlantVillage/Pepper__bell___healthy/00100ffa-095e-4881-aebf-61fe5af7226e___JR_HL 7886.JPG'
print(predict_class_names(model,img,class_indices))

1/1 [==============================] - 0s 18ms/step
Pepper__bell___healthy


In [ ]:
model.save('plant_disease_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
